In [1]:
# !pip install unidecode


In [2]:
# !pip install beautifulsoup4
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import json
import unidecode

In [3]:
def get_page_data(cityName):
    try:
        url = 'https://www.raiffeisen.ch/oberes-rheintal/de/privatkunden/hypotheken/hypopedia/gemeindeinfo.'+cityName+'.html'
        source = requests.get(url).text
        soup = BeautifulSoup(source,'lxml')
        category_data = soup.find_all("div",{"class":"fsBarChart__base"})
        if(len(category_data)==0):
            return 'error'
        return category_data
    except Exception as e:
        print(e)
        return 'error'

In [4]:
def get_first_category_data(category_data):
    try:  
        Einfamilienhauser = json.loads(category_data[0]['data-series'])
        Stockwerkeigentum = json.loads(category_data[1]['data-series'])
        Mietwohnungen = json.loads(category_data[2]['data-series'])
        return [Einfamilienhauser,Stockwerkeigentum,Mietwohnungen]
    except Exception as e:
        print(e)
        return "error"

In [5]:
def get_second_category_data(category_data):
    try:
        Angebot_Nachfrage = json.loads(category_data[7]['data-series'])
        Angebot_Nachfrage_nach_Typen = json.loads(category_data[8]['data-series'])
        return [Angebot_Nachfrage,Angebot_Nachfrage_nach_Typen]
    except Exception as e:
        print(e)
        return "error"

In [6]:
def get_third_category_data(category_data):
    try:
        bau = json.loads(category_data[14]['data-series'])
        return bau
    except Exception as e:
        print(e)
        return "error"

In [7]:
def get_fourth_category_data(cityName):
    try:
        url = 'https://www.raiffeisen.ch/oberes-rheintal/de/privatkunden/hypotheken/hypopedia/gemeindeinfo.'+cityName+'.html'
        source = requests.get(url).text
        soup = BeautifulSoup(source,'lxml')
        raw_names = soup.find('section',{"class":"factsheet__timeToCentre"}).find_all('a',{'class':"fsIconlist__cityName"})
        raw_values = soup.find('section',{"class":"factsheet__timeToCentre"}).find_all('td',{'class':"fsIconlist__itemData"})
        data = {} # {'name':[v1,v2]}
       
        counter= 0 
        v1 = ""
        v2 = ""
        for x in range(len(raw_names)):
                for y in range(0,1):
                    v1 = raw_values[counter].text
                    v2 = raw_values[counter+1].text
                
                counter = counter+2
                data[raw_names[x].text] = [int(v1),int(v2)]
        return data
    except Exception as e:
        print(e)
        return "error"

In [8]:
df_cat_1 = pd.DataFrame(columns = ['dge', 'Type', 'Region','Min','Max'])
df_cat_2 = pd.DataFrame(columns = ['dge', 'Topology', 'Region','Number Suchabo'])
df_cat_3 = pd.DataFrame(columns = ['dge', 'Topology', 'Size','Percentage'])
df_cat_4 = pd.DataFrame(columns = ['dge', 'Topology', 'Region','Nuber'])
df_cat_5 = pd.DataFrame(columns = ['dge', 'Name','value-1','value-2'])

In [9]:
cities = pd.read_excel('./City_List.xlsx')
sample = cities
for x in sample['dge_name'].tolist():
    city = unidecode.unidecode(x)
    try:
        category_data = get_page_data(city)
        d_1 = get_first_category_data(category_data)
        d_2 = get_second_category_data(category_data)
        d_3 = get_third_category_data(category_data)
        d_4 = get_fourth_category_data(city)
        if(type(d_1)==str):
            print("error")
            continue
        if(type(d_2)==str):
            print("error")
            continue
        if(type(d_3)==str):
            print("error")
            continue
        if(type(d_4)==str):
            print("error")
            continue
        Einfamilienhauser = d_1[0]
        Stockwerkeigentum = d_1[1]
        Mietwohnungen = d_1[2]
    
    
        #--------------------------------------------------------#
        for x in range(len(Einfamilienhauser['series'])):
            df_cat_1 = df_cat_1.append({'dge' : city, 'Type':'Einfamilienhauser' ,'Region':Einfamilienhauser['series'][x]['name'] , "Min":Einfamilienhauser['series'][x]['data'][0]['low'],"Max":Einfamilienhauser['series'][x]['data'][0]['high'] },ignore_index = True)
        
        for x in range(len(Stockwerkeigentum['series'])):
            df_cat_1 = df_cat_1.append({'dge' : city, 'Type':'Stockwerkeigentum' ,'Region':Stockwerkeigentum['series'][x]['name'] , "Min":Stockwerkeigentum['series'][x]['data'][0]['low'],"Max":Stockwerkeigentum['series'][x]['data'][0]['high'] },ignore_index = True)
    
        for x in range(len(Einfamilienhauser['series'])):
            df_cat_1 = df_cat_1.append({'dge' : city, 'Type':'Mietwohnungen' ,'Region':Mietwohnungen['series'][x]['name'] , "Min":Mietwohnungen['series'][x]['data'][0]['low'],"Max":Mietwohnungen['series'][x]['data'][0]['high'] },ignore_index = True)
        #--------------------------------------------------------#
    
        #---------------------------------------------------------#
        Angebot_Nachfrage = d_2[0]
        counter = 0
        for x in Angebot_Nachfrage['categories']:
            df_cat_2 = df_cat_2.append({'dge':city,'Topology':Angebot_Nachfrage['series'][0]['name'],'Region':x,'Number Suchabo':Angebot_Nachfrage['series'][0]['data'][counter]},ignore_index=True)
            df_cat_2 = df_cat_2.append({'dge':city,'Topology':Angebot_Nachfrage['series'][1]['name'],'Region':x,'Number Suchabo':Angebot_Nachfrage['series'][1]['data'][counter]},ignore_index=True)
            df_cat_2 = df_cat_2.append({'dge':city,'Topology':Angebot_Nachfrage['series'][2]['name'],'Region':x,'Number Suchabo':Angebot_Nachfrage['series'][2]['data'][counter]},ignore_index=True)
            counter = counter + 1
        #---------------------------------------------------------#
    
        #----------------------------------------------------------#
        Angebot_Nachfrage_nach_Typen = d_2[1]
        counter = 0
        for x in Angebot_Nachfrage_nach_Typen['categories']:
            for y in range(0,2):
                df_cat_3 = df_cat_3.append({'dge':city,'Topology':Angebot_Nachfrage_nach_Typen['series'][y]['name'],'Size':x,'Percentage':Angebot_Nachfrage_nach_Typen['series'][y]['data'][counter]},ignore_index=True)
            counter = counter + 1
        #----------------------------------------------------------#
    
        #-----------------------------------#
        bau = d_3
        counter = 0
        for x in bau['categories']:
            df_cat_4 = df_cat_4.append({'dge':city,'Topology':x,'Region':bau['series'][0]['name'],'Nuber':bau['series'][0]['data'][counter]},ignore_index=True)
            df_cat_4 = df_cat_4.append({'dge':city,'Topology':x,'Region':bau['series'][1]['name'],'Nuber':bau['series'][1]['data'][counter]},ignore_index=True)
            df_cat_4 = df_cat_4.append({'dge':city,'Topology':x,'Region':bau['series'][2]['name'],'Nuber':bau['series'][2]['data'][counter]},ignore_index=True)
            counter = counter + 1
        #------------------------------------#
    
        #-------------------------------------#
        for x in d_4.keys():
            df_cat_5 = df_cat_5.append({'dge' : city,'Name':x,'value-1':d_4[x][0],'value-2':d_4[x][1]},ignore_index=True)
        #-------------------------------------#
    
    except Exception as e:
        print(e)

string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out

'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
er

string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out

'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
er

string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out of range
string index out of range
'NoneType' object has no attribute 'find_all'
error
string indices must be integers
string index out

In [10]:
writer = pd.ExcelWriter('./outputFile.xlsx', engine='xlsxwriter')
# Write each dataframe to a different worksheet.
df_cat_1.to_excel(writer, sheet_name='Immobilienpreise')
df_cat_2.to_excel(writer, sheet_name='A&N Mietwohnungen')
df_cat_3.to_excel(writer, sheet_name='A&N nach Typen')
df_cat_4.to_excel(writer, sheet_name='Bau')
df_cat_5.to_excel(writer, sheet_name='Mobilitat')

# Close the Pandas Excel writer and output the Excel file.
writer.save()